# Iris data Softmax with Early stopping

## Initialize

In [117]:
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(2042)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)


## Fetch Data

In [118]:
# Fetch spam data

from sklearn import datasets
iris = datasets.load_iris()
list(iris.keys())

['data',
 'target',
 'frame',
 'target_names',
 'DESCR',
 'feature_names',
 'filename']

In [119]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

## Prepare Data

In [120]:
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = iris["target"]

In [121]:
#add the bias term for every instance (X0 = 1)
X_with_bias = np.c_[np.ones([len(X), 1]), X]


In [122]:
#convert y to one_hot vector
def to_one_hot(y_input):
    n_classes = y_input.max() + 1
    m = len(y_input)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y_input] = 1
    return Y_one_hot

In [123]:
def split_manual(X_input, y_input, validation_ratio, test_ratio):
    total_size = len(X_input)
    test_size = int(total_size * test_ratio)
    validation_size = int(total_size * validation_ratio)
    train_size = total_size - test_size - validation_size

    rnd_indices = np.random.permutation(total_size)
    X_train = X_input[rnd_indices[:train_size]]
    X_valid = X_input[rnd_indices[train_size:-test_size]]
    X_test = X_input[rnd_indices[-test_size:]]

    y_train = y_input[rnd_indices[:train_size]]
    y_valid = y_input[rnd_indices[train_size:-test_size]]
    y_test = y_input[rnd_indices[-test_size:]]

    return X_train, X_valid, X_test, y_train, y_valid, y_test



In [124]:
#split data to train, valid and test
X_train, X_valid, X_test, y_train, y_valid, y_test = split_manual(X_with_bias, y, 0.2, 0.2)

In [125]:
y_train_one_hot = to_one_hot(y_train)
y_valid_one_hot = to_one_hot(y_valid)
y_test_one_hot = to_one_hot(y_test)

## Train Model

In [126]:
## Define softmax function
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [127]:
def softmax_train(X,y, eta=0.01, n_iterations=5001, epsilon=1e-7):
    m = len(X)
    n_inputs = X.shape[1] # == 3 (2 features plus the bias term)
    n_outputs = len(np.unique(y, axis=0))   # == 3 (3 iris classes)
    Theta = np.random.randn(n_inputs, n_outputs)
    for iteration in range(n_iterations):
        logits = X.dot(Theta)
        Y_proba = softmax(logits)
        loss = -np.mean(np.sum(y * np.log(Y_proba + epsilon), axis=1))
        error = Y_proba - y
        if iteration % 500 == 0:
            print(iteration, loss)
        gradients = 1/m * X.T.dot(error)
        Theta = Theta - eta * gradients
    
    return Theta

In [128]:
#Theta = softmax_train(X_train, y_train_one_hot)

In [140]:
def softmax_proba(X_input, Theta_input):
    logits = X_input.dot(Theta_input)
    return softmax(logits)
    

def softmax_predict(X_input, Theta_input):
    Y_proba = softmax_proba(X_input, Theta_input)
    return np.argmax(Y_proba, axis=1)   

def softmax_accuracy(y_true, y_pred):
    return np.mean(y_pred == y_true)


In [141]:
Theta0 = softmax_train(X_train, y_train_one_hot)
y_predict0 = softmax_predict(X_valid, Theta0)
print(softmax_accuracy(y_valid, y_predict0))

0 3.2078708887337366
500 0.9102655715933011
1000 0.7245981143541181
1500 0.6195736452392622
2000 0.5544764557510471
2500 0.510121522324934
3000 0.4775255492382668
3500 0.4521715148580381
4000 0.4315976459646628
4500 0.41436168630147824
5000 0.39956523349383655
0.9666666666666667


## Train Model with Regularization

In [132]:
def softmax_train_reg(X,y, eta=0.01, n_iterations=5001, epsilon=1e-7, alpha=0):
    reg_value = 0
    m = len(X)
    n_inputs = X.shape[1] # == 3 (2 features plus the bias term)
    n_outputs = len(np.unique(y, axis=0))   # == 3 (3 iris classes)
    Theta = np.random.randn(n_inputs, n_outputs)
    for iteration in range(n_iterations):
        logits = X.dot(Theta)
        Y_proba = softmax(logits)
        loss = -np.mean(np.sum(y * np.log(Y_proba + epsilon), axis=1))

        #calculate regularization
        if (alpha > 0):
            l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
            loss = loss + alpha * l2_loss
            reg_value = np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]

        error = Y_proba - y
        if iteration % 500 == 0:
            print(iteration, loss)
        
        gradients = 1/m * X_train.T.dot(error) + reg_value
        Theta = Theta - eta * gradients
    
    return Theta

In [177]:
Theta1 = softmax_train_reg(X_train, y_train_one_hot)
y_predict1 = softmax_predict(X_valid, Theta1)
print(softmax_accuracy(y_valid, y_predict1))

0 5.724334119069223
500 0.8486894750536992
1000 0.6852227121507914
1500 0.5922028145237946
2000 0.534401132587006
2500 0.49496352341377414
3000 0.4659244611663126
3500 0.44326242819834444
4000 0.4247862232100028
4500 0.40921651470821874
5000 0.39576088473524795
0.9333333333333333


In [134]:
Theta2 = softmax_train_reg(X_train, y_train_one_hot, eta=0.1, alpha=0.01, n_iterations=5001)
y_predict2 = softmax_predict(X_valid, Theta2)
print(softmax_accuracy(y_valid, y_predict2))


0 5.1960577948669044
500 0.43488021031204266
1000 0.36229502865996227
1500 0.3245403372526724
2000 0.30051903066813007
2500 0.28399519364079534
3000 0.27209348359164676
3500 0.2632401967817035
4000 0.2564915989643679
4500 0.2512468493539844
5000 0.24710620351627532
0.9666666666666667


## Train Model with Early Stopping

In [143]:
def softmax_train_reg_stop(X,y, early_stop = False, X_valid = None, y_valid = None, y_valid_one_hot=None, eta=0.01, n_iterations=5001, epsilon=1e-7, alpha=0, ):
    reg_value = 0
    m = len(X)
    n_inputs = X.shape[1] # == 3 (2 features plus the bias term)
    n_outputs = len(np.unique(y, axis=0))   # == 3 (3 iris classes)
    Theta = np.random.randn(n_inputs, n_outputs)
    best_loss = np.infty

    for iteration in range(n_iterations):
        logits = X.dot(Theta)
        Y_proba = softmax(logits)
        loss = -np.mean(np.sum(y * np.log(Y_proba + epsilon), axis=1))

        #calculate regularization
        if (alpha > 0):
            l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
            loss = loss + alpha * l2_loss
            reg_value = np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]

        error = Y_proba - y
        gradients = 1/m * X_train.T.dot(error) + reg_value
        Theta = Theta - eta * gradients

        if iteration % 500 == 0:
            print(iteration, loss)

        if early_stop:
            y_proba = softmax_proba(X_valid, Theta)
            xentropy_loss = -np.mean(np.sum(y_valid_one_hot * np.log(y_proba + epsilon), axis=1))
            l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
            loss = xentropy_loss + alpha * l2_loss
            if loss < best_loss:
                best_loss = loss
            else:
                print(iteration - 1, best_loss)
                print(iteration, loss, "early stopping!")
                return Theta

    return Theta

In [151]:
Theta1 = softmax_train_reg_stop(X_train, y_train_one_hot)
y_predict1 = softmax_predict(X_valid, Theta1)
print(softmax_accuracy(y_valid, y_predict1))

0 2.0622969029233422
500 0.8699865175672002
1000 0.6991894533616019
1500 0.6009046169921253
2000 0.5400563760622615
2500 0.49887389844420527
3000 0.468804047568738
3500 0.44551217441689067
4000 0.426640342163956
4500 0.4108171013843986
5000 0.3971969387689109
0.9333333333333333


In [147]:
Theta2 = softmax_train_reg_stop(X_train, y_train_one_hot, eta=0.1, alpha=0.01, n_iterations=5001)
y_predict2 = softmax_predict(X_valid, Theta2)
print(softmax_accuracy(y_valid, y_predict2))

0 4.20473651918448
500 0.4348396152428904
1000 0.36228045170776024
1500 0.3244485923105366
2000 0.30039390882562095
2500 0.2838692403156974
3000 0.27198124739028
3500 0.2631455213784823
4000 0.2564136436119871
4500 0.2511832335942712
5000 0.2470543542440645
0.9666666666666667


In [146]:
Theta3 = softmax_train_reg_stop(X_train, y_train_one_hot, early_stop = True, X_valid = X_valid, y_valid = y_valid, y_valid_one_hot=y_valid_one_hot, eta=0.1, alpha=0.1, n_iterations=5001)
y_predict3 = softmax_predict(X_valid, Theta3)
print(softmax_accuracy(y_valid, y_predict3))

0 3.248603418763468
500 0.5236482529721027
1000 0.5012026401848378
1500 0.4938085566634075
2000 0.4909411221337844
2500 0.48974810711102384
2614 0.5325466937764207
2615 0.5325466948333359 early stopping!
1.0
